In [48]:
# relative to this notebook,  adding src folder to sys.path
src_path = os.path.abspath(os.path.join('..'))
src = os.path.join(src_path,"src")
if src not in sys.path:
    sys.path.append(src)

In [49]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# https://towardsdatascience.com/a-friendly-introduction-to-text-clustering-fa996bcefd04

In [62]:
from d01_data.dataSlicer import data_slicer

# define the path and type of input file and out put file
data_in = os.path.join(src_path, 'data','02_intermediate','ArticleDataset.json')
data_out = os.path.join(src_path, 'data','02_intermediate','Article1000.csv')

# call the data slicer function
data_slicer(data_in,data_out,1000)

In [63]:
#data_in = os.path.join(src_path, 'data','02_intermediate','ArticleDataset.json')
#data_in = os.path.join(src_path, 'data','02_intermediate','article_20.csv')
data_in = os.path.join(src_path, 'data','02_intermediate','Article1000.csv')

#articles = pd.read_json(data_in)
articles = pd.read_csv(data_in, index_col=0)

FileNotFoundError: [Errno 2] File /home/jishnu/Desktop/Rokin_Dev/data/02_intermediate/Article1000.csv does not exist: '/home/jishnu/Desktop/Rokin_Dev/data/02_intermediate/Article1000.csv'

In [52]:
# name the columns
articles.columns = ['date','heading','content','link','empty']

# cut down columns
articles = articles[['heading','content']]
articles.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   heading  20 non-null     object
 1   content  20 non-null     object
dtypes: object(2)
memory usage: 480.0+ bytes


In [61]:
vec = TfidfVectorizer(stop_words="english")
vec.fit(articles.content.values)
features = vec.transform(articles.content.values)        # <--- feature matrix

# try dim reduction using LSI
# TODO

# DBSCAN model, tune the parameter
dbscan = DBSCAN(eps=0.5, min_samples=2)
db1 = dbscan.fit(features)

labels = db1.labels_

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print("number of clusters:",n_clusters)

# identify core points
core_samples = np.zeros_like(labels,dtype =bool)
core_samples[dbscan.core_sample_indices_] = True
print(core_samples)

# metric, Sillhouette score or BIC

number of clusters: 0
[False False False False False False False False False False False False
 False False False False False False False False]
